In [ ]:
!sudo apt-get update && sudo apt-get install -y libwoff1 libopus0 libwebp6 libwebpdemux2 libenchant1c2a libgudev-1.0-0 libsecret-1-0 libhyphen0 libgdk-pixbuf2.0-0 libegl1 libnotify4 libxslt1.1 libevent-2.1-7 libgles2 libvpx6 libxcomposite1 libatk1.0-0 libatk-bridge2.0-0 libepoxy0 libgtk-3-0 libharfbuzz-icu0

In [ ]:
# !pip install "crawl4ai @ git+https://github.com/unclecode/crawl4ai.git"
# !pip install "crawl4ai @ git+https://github.com/unclecode/crawl4ai.git@staging"
!pip install crawl4ai
!pip install nest-asyncio
!playwright install

In [ ]:
import asyncio
import nest_asyncio
from crawl4ai import AsyncWebCrawler
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy, LLMExtractionStrategy
import json
import time
from pydantic import BaseModel, Field

nest_asyncio.apply()

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-proj-xxxx'

In [ ]:
import os
from google.colab import userdata


class OpenAIModelFee(BaseModel):
    player_name: str = Field(..., description="Name of the player")
    elo_score: str = Field(..., description="Elo Score of the player")
    games_number: str = Field(..., description="Number of games of the player")


async def extract_openai_fees():
    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(
            url='https://lichess.org/broadcast/fide-world-youth-chess-championships-2024--o18/round-1/0SPgwmAM#players',
            word_count_threshold=1,
            extraction_strategy=LLMExtractionStrategy(
                provider="openai/gpt-4o-mini-2024-07-18", api_token=os.getenv('OPENAI_API_KEY'),
                schema=OpenAIModelFee.schema(),
                extraction_type="schema",
                instruction="""From the crawled content, extract all mentioned player names along with their scores and games.
                Do not miss any models in the entire content. One extracted model JSON format should look like this:
                {"player_name": "Abdul M", "elo_score": "1400", "games_number": "0"}."""
            ),
            bypass_cache=True
        )
        print((result.extracted_content))

# Uncomment the following line to run the OpenAI extraction example
await extract_openai_fees()